In [5]:
import logging
import boto3
from botocore.exceptions import ClientError
import os
import json


def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

def set_bucket_public(bucket_name):
    """Set the bucket policy to make all objects in the bucket publicly readable

    :param bucket_name: Bucket to set the policy on
    :return: True if policy was set, else False
    """
    s3_client = boto3.client('s3')

    # Define the bucket policy
    bucket_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "PublicReadForAllObjects",
                "Effect": "Allow",
                "Principal": "*",
                "Action": "s3:GetObject",
                "Resource": f"arn:aws:s3:::{bucket_name}/*"
            }
        ]
    }

    # Convert the policy from JSON dict to string
    bucket_policy = json.dumps(bucket_policy)

    # Set the new policy
    try:
        s3_client.put_bucket_policy(Bucket=bucket_name, Policy=bucket_policy)
    except ClientError as e:
        logging.error(e)
        return False
    return True

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

def upload_directory(directory_name, bucket, s3_prefix=''):
    """Upload a directory to an S3 bucket

    :param directory_name: Directory to upload
    :param bucket: Bucket to upload to
    :param s3_prefix: S3 prefix for the uploaded files
    :return: True if directory was uploaded, else False
    """
    s3_client = boto3.client('s3')

    for root, dirs, files in os.walk(directory_name):
        for file in files:
            file_path = os.path.join(root, file)
            s3_path = os.path.relpath(file_path, directory_name)
            if s3_prefix:
                s3_path = os.path.join(s3_prefix, s3_path)

            try:
                s3_client.upload_file(file_path, bucket, s3_path)
                print(f'Successfully uploaded {file_path} to s3://{bucket}/{s3_path}')
            except ClientError as e:
                logging.error(e)
                return False
    return True

In [2]:
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  das-test-cube


Right now setting the bucket to public needs to be done manually...

In [3]:
# Define the directory and bucket
cube_dir = "/work/le837wmue-Rhone-download/cache/cube/DAS_test.zarr"
bucket = "das-up"

# Create the bucket
if create_bucket(bucket, "eu-north-1"):
    # Set the bucket policy to make it public
    if set_bucket_public(bucket):
        print(f'Bucket {bucket} is now public.')
    else:
        print(f'Failed to set the bucket policy for {bucket}.')

    # Upload the directory
    if upload_directory(cube_dir, bucket):
        print(f'Successfully uploaded directory {cube_dir} to bucket {bucket}.')
    else:
        print(f'Failed to upload directory {cube_dir} to bucket {bucket}.')
else:
    print(f'Failed to create bucket {bucket}.')

ERROR:root:An error occurred (AccessDenied) when calling the PutBucketPolicy operation: Access Denied


Failed to set the bucket policy for das-up.
Successfully uploaded /work/le837wmue-Rhone-download/cache/cube/DAS_test.zarr/.zgroup to s3://das-up/.zgroup
Successfully uploaded /work/le837wmue-Rhone-download/cache/cube/DAS_test.zarr/.zmetadata to s3://das-up/.zmetadata
Successfully uploaded /work/le837wmue-Rhone-download/cache/cube/DAS_test.zarr/.zattrs to s3://das-up/.zattrs
Successfully uploaded /work/le837wmue-Rhone-download/cache/cube/DAS_test.zarr/channel/0 to s3://das-up/channel/0
Successfully uploaded /work/le837wmue-Rhone-download/cache/cube/DAS_test.zarr/channel/.zarray to s3://das-up/channel/.zarray
Successfully uploaded /work/le837wmue-Rhone-download/cache/cube/DAS_test.zarr/channel/.zattrs to s3://das-up/channel/.zattrs
Successfully uploaded /work/le837wmue-Rhone-download/cache/cube/DAS_test.zarr/frequency/0 to s3://das-up/frequency/0
Successfully uploaded /work/le837wmue-Rhone-download/cache/cube/DAS_test.zarr/frequency/.zattrs to s3://das-up/frequency/.zattrs
Successfully u

In [4]:
s3 = boto3.client('s3')
response = s3.list_buckets()

print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  das-test-cube
  das-up
